In [147]:
from tensorflow import keras
from keras.applications import VGG16

conv_base = VGG16(weights = "imagenet",
include_top = False,
input_shape = (224, 224, 3))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [148]:
for layer in conv_base.layers:
    layer.trainable = False

In [149]:
import splitfolders
inputfolder = 'Final/Animals'
output = 'Final'
splitfolders.ratio(inputfolder, output = output, seed = 40, ratio = (0.8, 0.1, 0.1))

Copying files: 582 files [00:11, 52.22 files/s]


In [270]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0,
                                                             rotation_range = 10,
                                                             width_shift_range=0.1,
                                                             height_shift_range=0.1,
                                                             fill_mode = 'nearest',
                                                             shear_range=0.2, 
                                                             zoom_range=0.2, 
                                                             horizontal_flip = True,
                                                             vertical_flip = True)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)

In [271]:
trainGen = train_datagen.flow_from_directory(
    r'C:\Users\vidha\Machine Learning\Final\train',
    target_size = (224,224),
    batch_size = 32,
    shuffle = True,
    class_mode = 'categorical'
)

testGen = test_datagen.flow_from_directory(
    r'C:\Users\vidha\Machine Learning\Final\test',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

valGen = test_datagen.flow_from_directory(
    r'C:\Users\vidha\Machine Learning\Final\val',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 465 images belonging to 10 classes.
Found 60 images belonging to 10 classes.
Found 57 images belonging to 10 classes.


In [272]:
import numpy as np
def extractfeatures(sample_count, generator):
    i = 0
    batch_size = 32
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count, 10))
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1) * batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if ((i * batch_size % 1000) == 0 ):
            print("processed size =", i * batch_size)
        if i * batch_size >= sample_count:
            break
    return features, labels

In [273]:
train_features, train_labels = extractfeatures(465, trainGen)
test_features, test_labels = extractfeatures(60, testGen)
val_features, val_labels = extractfeatures(57, valGen)

1/1 [==============================] - 16s 16s/step


In [304]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(7, 7, 512)),
    keras.layers.Dense(256, activation = 'relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation = 'softmax'),
])

In [305]:
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [306]:
es = keras.callbacks.EarlyStopping(monitor = 'val_loss', verbose = 1, mode = 'min', patience = 5)

In [307]:
model.fit(train_features, train_labels, steps_per_epoch = 465 // 32, validation_data = (val_features, val_labels), validation_steps = 57 // 32, epochs = 10, callbacks = [es])

Epoch 1/10
14/14 [==============================] - 7s 211ms/step - loss: 2.5845 - accuracy: 0.0860 - val_loss: 2.1701 - val_accuracy: 0.2807
Epoch 2/10
14/14 [==============================] - 2s 151ms/step - loss: 2.0973 - accuracy: 0.2624 - val_loss: 2.0432 - val_accuracy: 0.4035
Epoch 3/10
14/14 [==============================] - 2s 158ms/step - loss: 1.8558 - accuracy: 0.4043 - val_loss: 1.9093 - val_accuracy: 0.3509
Epoch 4/10
14/14 [==============================] - 2s 147ms/step - loss: 1.6538 - accuracy: 0.4667 - val_loss: 1.8383 - val_accuracy: 0.3860
Epoch 5/10
14/14 [==============================] - 2s 140ms/step - loss: 1.4631 - accuracy: 0.5269 - val_loss: 1.7096 - val_accuracy: 0.4211
Epoch 6/10
14/14 [==============================] - 2s 145ms/step - loss: 1.2780 - accuracy: 0.6323 - val_loss: 1.7036 - val_accuracy: 0.2807
Epoch 7/10
14/14 [==============================] - 2s 167ms/step - loss: 1.1466 - accuracy: 0.6774 - val_loss: 1.5776 - val_accuracy: 0.5088
Epoch 

In [308]:
model.evaluate(test_features, test_labels, steps = 60//32)

1/1 [==============================] - 1s 749ms/step - loss: 1.4920 - accuracy: 0.5167


[1.492045283317566, 0.5166666507720947]

In [309]:
model.evaluate(train_features, train_labels, steps = 465 // 32)

14/14 [==============================] - 0s 21ms/step - loss: 0.6163 - accuracy: 0.9247


[0.6162633299827576, 0.9247311949729919]